# WeatherPy
----

In [1]:
# Dependencies and Setup
# import os
import matplotlib.pyplot as plt
import pandas as pd
# import numpy as np
import requests
# import time
# import csv
from datetime import datetime
from citipy import citipy
from api_keys import api_key

## Generate Cities List

In [5]:
# Set ranges of potential latitudes and longitudes
lat_set = (-90, 90)
long_set = (-180, 180)

# create empty lists for coordinates, cities/searchable city names, and country codes
lat_longs = []
cities = []
search_cities = [] # replaces each :space: in city name with :+: symbol allowing for usage in API web address
countries = []

# Create a set of random coordinates (min 500 usable results)
lats = np.random.uniform(low = -90.0000, high = 90.0000, size = 2000)
longs = np.random.uniform(low = -180.0000, high = 180.0000, size = 2000)
lat_longs = zip(lats, longs)

# Identify nearest city for each set of coordinates, add only if not already in list
for lat_long in lat_longs:
    city_name = citipy.nearest_city(lat_long[0], lat_long[1]).city_name
    country_code = citipy.nearest_city(lat_long[0], lat_long[1]).country_code
    if city_name not in cities:
        cities.append(city_name)
        countries.append(country_code)

# Replace spaces in multi-word city names with '+' so url doesn't 'break' at space in API address
for city in cities:
        readble_city = city.replace(' ', '+') # may alternately use '+' OR '%20' - both work for this API!
        search_cities.append(readble_city)

# generate searchable item list for API
search_list = list(zip(search_cities, countries))

## Perform API Calls

In [6]:
# Base API url for calls
url = 'http://api.openweathermap.org/data/2.5/weather?appid='
units = 'imperial'

In [7]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
#                                API ADDRESS & JSON DATA REVIEW BLOCK                                 #
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

#  review data retrieved in call

# #  test variables - replace both of these with proper variables in iterable code
# city = 'Orlando'
# country = 'US'

# city_url = url + api_key + '&units=' + units + '&q=' + city + ',' + country
# response = requests.get(city_url) 
# weather_data = response.json() 

# #   test dumps for reading json retrieval
# print(json.dumps(weather_data, indent=3, sort_keys=True))

# #   test URL retrieval
# print(response.url)

In [8]:
city_cap = [] # duplicates info from cities variable, but in a nicer format for df (proper capitalization)
country_cap = [] # duplicates info from countries variable, but in a nicer format for df (proper capitalization)
lat = []
temp = []
humidity = []
cloudiness = []
wind_speed = []
dates = []
failure_count = []


print('''
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
=                                                 Processing Records                                                  =
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

''')

for call_count in range(len(search_list)):
    try:
        city = search_list[call_count][0]
        country = search_list[call_count][1]
        
        city_url = (url + api_key + '&units='+ units + '&q=' + city + ',' + country)
        weather_data = requests.get(city_url).json() 

        if weather_data["cod"] == '404':
            call_count += 1
            failure_count.append(f'City Call {call_count}')
            print(f'City Call {call_count}:  {str.title(city)}, {str.upper(country)} ***** FAILED: City Not Found *****')
        elif weather_data["main"]["humidity"] >= 101:
            call_count += 1
            failure_count.append(f'City Call {call_count}')
            print(f'City Call {call_count}: {city_log}, {country_log} ***** DATA OUT OF BOUNDS: City Not Included *****')
        else:
            city_cap.append(weather_data["name"])
            city_log = weather_data["name"]
            country_cap.append(weather_data["sys"]["country"])
            country_log = weather_data["sys"]["country"]
            lat.append(weather_data["coord"]["lat"])
            temp.append(weather_data["main"]["temp_max"])
            humidity.append(weather_data["main"]["humidity"])
            cloudiness.append(weather_data["clouds"]["all"])
            wind_speed.append(weather_data["wind"]["speed"])
            timestamp = datetime.fromtimestamp(weather_data['dt']).strftime('%m/%d/%Y - %I:%M %p')
            dates.append(timestamp)

            call = (f'City Call {call_count + 1}:  {city_log}, {country_log}')
            print(call)

        call_count += 1

    except:
        print('= + = + = + = + = + = +     ERROR     + = + = + = + = + = + =')
    pass  


print('''
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
=                                              Data Collection Complete!                                              =
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
''')

# quick review/validation of lists 
print(f'''
{len(search_list)} random, unique cities generated.

{len(failure_count)} FAILURES TO CALL:
-----------------------------------------------------------------------------------------------------------------------
{failure_count}


Complete data will have {len(search_list) - len(failure_count)} items.

Item Count:
{len(city_cap)} Cities
{len(country_cap)} Countries
{len(lat)} Latitudes
{len(temp)} Temperatures
{len(humidity)} Humidity Percentages
{len(cloudiness)} Cloudiness Percentages
{len(wind_speed)} Wind Speeds
{len(dates)} Dates
''')


=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
=                                                 Processing Records                                                  =
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


City Call 1:  Kruisfontein, ZA
City Call 2:  Parsons, US
City Call 3:  Barrow, US
City Call 4:  Punta Arenas, CL
City Call 5:  Dingzhou, CN
City Call 6:  Iqaluit, CA
City Call 7:  Talnakh, RU
City Call 8:  Cidreira, BR
City Call 9:  Sukhobezvodnoye, RU
City Call 10:  Lima, PE
City Call 11:  Hobart, AU
City Call 12:  Rikitea, PF
City Call 13:  Kapaa, US
City Call 14:  Esperance, AU
City Call 15:  Mahebourg, MU
City Call 16:  Ilulissat, GL
City Call 17:  Kahului, US
City Call 18:  Evensk, RU
City Call 19:  Mahon, ES ***** FAILED: City Not Found *****
City Call 20:  Busselton, AU
City Call 21:  San Patricio, MX
City Call 22:  Hilo, 

City Call 234:  Sitka, US
City Call 235:  Svetlogorsk, RU
City Call 236:  Binay, PH
City Call 237:  Biak, ID
City Call 238:  Honningsvag, NO
City Call 239:  Lavrentiya, RU
City Call 240:  Sokoni, TZ ***** FAILED: City Not Found *****
City Call 241:  Vallenar, CL
City Call 242:  Hunza, PK ***** FAILED: City Not Found *****
City Call 243:  Ribeira Grande, PT
City Call 244:  Marsa Matruh, EG
City Call 245:  Butaritari, KI
City Call 246:  Cherskiy, RU
City Call 247:  Bethel, US
City Call 248:  Yulara, AU
City Call 249:  Tasiilaq, GL
City Call 250:  Ucluelet, CA
City Call 251:  Naberera, TZ
City Call 252:  Tateyama, JP
City Call 253:  Umtata, ZA
City Call 254:  Douentza, ML
City Call 255:  Wismar, DE
City Call 256:  Dujuma, SO ***** FAILED: City Not Found *****
City Call 257:  Mangit, UZ
City Call 258:  Ormara, PK
City Call 259:  Carnarvon, AU
City Call 260:  Mys+Shmidta, RU ***** FAILED: City Not Found *****
City Call 261:  Quang Ngai, VN
City Call 262:  Calatayud, ES
City Call 263:  Zyrya

City Call 471:  Santo Angelo, BR
City Call 472:  Kalmunai, LK
City Call 473:  Mehamn, NO
City Call 474:  Umm Kaddadah, SD
City Call 475:  Ponta Delgada, PT
City Call 476:  La Rioja, AR
City Call 477:  Dongsheng, CN
City Call 478:  Gao, ML
City Call 479:  Lagoa, PT
City Call 480:  Ambilobe, MG
City Call 481:  Cadillac, US
City Call 482:  Valky, UA
City Call 483:  Achisay, KZ ***** FAILED: City Not Found *****
City Call 484:  Tuatapere, NZ
City Call 485:  Grindavik, IS
City Call 486:  Kongolo, CD
City Call 487:  Phonhong, LA
City Call 488:  Gamba, GA
City Call 489:  Ovruch, UA
City Call 490:  Nha Trang, VN
City Call 491:  Sarh, TD
City Call 492:  Benton Harbor, US
City Call 493:  Kuala Kedah, MY
City Call 494:  Uslar, DE
City Call 495:  San Ramon, BO
City Call 496:  Challapata, BO
City Call 497:  Forio, IT
City Call 498:  Virginia Beach, US
City Call 499:  Karratha, AU
City Call 500:  Adamas, GR
City Call 501:  Aldan, RU
City Call 502:  Anloga, GH
City Call 503:  Alashtar, IR ***** FAILE

City Call 711:  Dyersburg, US
City Call 712:  Hami, CN
City Call 713:  Lukoyanov, RU
City Call 714:  Itoman, JP
City Call 715:  Glotovka, RU
City Call 716:  Jatibonico, CU
City Call 717:  Suluova, TR
City Call 718:  Miri, MY
City Call 719:  Nuevo Laredo, MX
City Call 720:  Ndele, CF
City Call 721:  Sumbawa, ID ***** FAILED: City Not Found *****
City Call 722:  Dukat, RU
City Call 723:  Am Timan, TD
City Call 724:  Pokrovsk, RU
City Call 725:  San Juan, US
City Call 726:  Tottori, JP
City Call 727:  Biliran, PH
City Call 728:  Varsag, RO
City Call 729:  Naryan-Mar, RU
City Call 730:  Moose Factory, CA
City Call 731:  Chekhov, RU
City Call 732:  Kurayoshi, JP
City Call 733:  Alta, NO
City Call 734:  Poplar Bluff, US
City Call 735:  Temple, US
City Call 736:  Mullaitivu, LK ***** FAILED: City Not Found *****
City Call 737:  Aquin, HT
City Call 738:  Stornoway, GB
City Call 739:  Conakry, GN
City Call 740:  Cockburn Town, BS
City Call 741:  Vila Franca do Campo, PT
City Call 742:  Galgani,

## Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
city_weather_df = (pd.DataFrame({'City': city_cap, 
                                 'Country': country_cap, 
                                 'Latitude': lat, 
                                 'Temperature (F)': temp, 
                                 'Humidity (%)': humidity, 
                                 'Cloudiness (%)': cloudiness, 
                                 'Wind Speed (mph)': wind_speed, 
                                 'Date - Time Reported': dates}))

# variable for naming convention of output files and DATE in plot titles: represents date the information is called
called_on = datetime.now().strftime('%B %d, %Y')

city_weather_df.head()

In [ ]:
city_weather_df.count()
city_weather_df.to_csv((f'Weather Data {called_on}.csv'), index=False)

## Plotting the Data

### Temperature Plot

In [ ]:
plt.figure(figsize = (15, 10))

plt.scatter(city_weather_df['Latitude'], city_weather_df['Temperature (F)'], 
            marker = 'o', 
            c = city_weather_df['Temperature (F)'],
            edgecolor = 'teal', 
            alpha = 0.75)

# Title the plot.
plt.title((f'Latitude vs Temperature\n({called_on})'), fontsize = 30)

# Label and limits for x-axis.
plt.xlabel('Location: Degrees of Latitude', fontsize = 16)
plt.xlim (-90, 90)

# Label and limits for y-axis.
plt.ylabel('Temperature: °Fahrenheit', fontsize = 16)
plt.ylim (20, 120)

# Add light grid with axis line for equator
plt.grid(color = 'silver', linestyle = '-.', linewidth = 0.5)
plt.axvline(linewidth = 2, color = 'coral')

# Show colorbar
plt.colorbar()

plt.savefig((f'Temperature {called_on}.png'))
plt.show()

In [ ]:
plt.figure(figsize = (9, 9))

plt.scatter(city_weather_df['Temperature (F)'], city_weather_df['Latitude'], 
            marker = 'o', 
            c = city_weather_df['Temperature (F)'],
            edgecolor = 'teal', 
            alpha = 0.75)

# Title the plot.
plt.title((f'Temperatures by Latitude\n({called_on})'), fontsize = 23)

# Label and limits for x-axis.
plt.ylabel('Location: Degrees of Latitude', fontsize = 16)
plt.ylim (-90, 90)

# Label and limits for y-axis.
plt.xlabel('Temperature: °Fahrenheit', fontsize = 16)
plt.xlim (20, 120)

# Add light grid with axis line for equator and notations for North and South
plt.grid(color = 'silver', linestyle = '-.', linewidth = 0.5)
plt.axhline(linewidth = 2, color = 'coral')
plt.text (121, -1.25, 'Equator', fontsize = 13, color = 'coral', style = 'italic')
plt.text (121, 80, 'North', fontsize = 13, color = 'navy')
plt.text (121, -83, 'South', fontsize = 13, color = 'navy')

# Show colorbar
plt.colorbar(orientation = 'horizontal')
plt.savefig((f'Temperature by Latitude {called_on}.png'))

plt.show()

### Humidity Plot

In [ ]:
plt.figure(figsize = (15, 10))

plt.scatter(city_weather_df['Latitude'], city_weather_df['Humidity (%)'], 
            marker = 'o', 
            c = city_weather_df['Humidity (%)'],
            edgecolor = 'teal', 
            alpha = 0.75)

# Title the plot.
plt.title((f'Latitude vs Humidity\n({called_on})'), fontsize = 30)

# Label and limits for x-axis.
plt.xlabel('Location: Degrees of Latitude', fontsize = 16)
plt.xlim (-90, 90)

# Label and limits for y-axis.
plt.ylabel('Percent Humidity', fontsize = 16)
plt.ylim (-2, 102)

# Add light grid with axis line for equator
plt.grid(color = 'silver', linestyle = '-.', linewidth = 0.5)
plt.axvline(linewidth = 2, color = 'coral')

# Show colorbar
plt.colorbar()
plt.savefig((f'Humidity {called_on}.png'))

plt.show()

In [ ]:
plt.figure(figsize = (9, 9))

plt.scatter(city_weather_df['Humidity (%)'], city_weather_df['Latitude'], 
            marker = 'o', 
            c = city_weather_df['Humidity (%)'],
            edgecolor = 'teal', 
            alpha = 0.75)

# Title the plot.
plt.title((f'Humidity by Latitude\n({called_on})'), fontsize = 23)

# Label and limits for x-axis.
plt.ylabel('Location: Degrees of Latitude', fontsize = 16)
plt.ylim (-90, 90)

# Label and limits for y-axis.
plt.xlabel('Percent Humidity', fontsize = 16)
plt.xlim (-2, 102)

# Add light grid with axis line for equator and notations for North and South
plt.grid(color = 'silver', linestyle = '-.', linewidth = 0.5)
plt.axhline(linewidth = 2, color = 'coral')
plt.text (103, -1.25, 'Equator', fontsize = 13, color = 'coral', style = 'italic')
plt.text (103, 80, 'North', fontsize = 13, color = 'navy')
plt.text (103, -83, 'South', fontsize = 13, color = 'navy')

# Show colorbar
plt.colorbar(orientation = 'horizontal')
plt.savefig((f'Humidity by Latitude {called_on}.png'))

plt.show()

### Cloudiness Plot

In [ ]:
plt.figure(figsize = (15, 10))

plt.scatter(city_weather_df['Latitude'], city_weather_df['Cloudiness (%)'], 
            marker = 'o', 
            c = city_weather_df['Cloudiness (%)'],
            edgecolor = 'teal', 
            alpha = 0.75)

# Title the plot.
plt.title((f'Latitude vs Cloudiness\n({called_on})'), fontsize = 30)

# Label and limits for x-axis.
plt.xlabel('Location: Degrees of Latitude', fontsize = 16)
plt.xlim (-90, 90)

# Label and limits for y-axis.
plt.ylabel('Percent Cloudiness', fontsize = 16)
plt.ylim (-2, 102)

# Add light grid with axis line for equator
plt.grid(color = 'silver', linestyle = '-.', linewidth = 0.5)
plt.axvline(linewidth = 2, color = 'coral')

# Show colorbar
plt.colorbar()
plt.savefig((f'Cloudiness {called_on}.png'))

plt.show()

In [ ]:
plt.figure(figsize = (9, 9))

plt.scatter(city_weather_df['Cloudiness (%)'], city_weather_df['Latitude'], 
            marker = 'o', 
            c = city_weather_df['Cloudiness (%)'],
            edgecolor = 'teal', 
            alpha = 0.75)

# Title the plot.
plt.title((f'Cloudiness by Latitude\n({called_on})'), fontsize = 23)

# Label and limits for x-axis.
plt.ylabel('Location: Degrees of Latitude', fontsize = 16)
plt.ylim (-90, 90)

# Label and limits for y-axis.
plt.xlabel('Percent Cloudiness', fontsize = 16)
plt.xlim (-2, 102)

# Add light grid with axis line for equator and notations for North and South
plt.grid(color = 'silver', linestyle = '-.', linewidth = 0.5)
plt.axhline(linewidth = 2, color = 'coral')
plt.text (103, -1.25, 'Equator', fontsize = 13, color = 'coral', style = 'italic')
plt.text (103, 80, 'North', fontsize = 13, color = 'navy')
plt.text (103, -83, 'South', fontsize = 13, color = 'navy')

# Show colorbar
plt.colorbar(orientation = 'horizontal')
plt.savefig((f'Cloudiness by Latitude {called_on}.png'))

plt.show()

### Wind Speed Plot

In [ ]:
plt.figure(figsize = (15, 10))

plt.scatter(city_weather_df['Latitude'], city_weather_df['Wind Speed (mph)'], 
            marker = 'o', 
            c = city_weather_df['Wind Speed (mph)'],            
            edgecolor = 'teal', 
            alpha = 0.75)

# Title the plot.
plt.title((f'Latitude vs Wind Speed\n({called_on})'), fontsize = 30)

# Label and limits for x-axis.
plt.xlabel('Location: Degrees of Latitude', fontsize = 16)
plt.xlim (-90, 90)

# Label and limits for y-axis.
plt.ylabel('Wind Speed (mph)', fontsize = 16)
plt.ylim (-1, 31)

# Add light grid with axis line for equator
plt.grid(color = 'silver', linestyle = '-.', linewidth = 0.5)
plt.axvline(linewidth = 2, color = 'coral')

# Show colorbar
plt.colorbar()
plt.savefig((f'Wind Speed {called_on}.png'))

plt.show()

In [ ]:
plt.figure(figsize = (9, 9))

plt.scatter(city_weather_df['Wind Speed (mph)'], city_weather_df['Latitude'], 
            marker = 'o', 
            c = city_weather_df['Wind Speed (mph)'],            
            edgecolor = 'teal', 
            alpha = 0.75)

# Title the plot.
plt.title((f'Wind Speed by Latitude\n({called_on})'), fontsize = 23)

# Label and limits for x-axis.
plt.ylabel('Location: Degrees of Latitude', fontsize = 16)
plt.ylim (-90, 90)

# Label and limits for y-axis.
plt.xlabel('Wind Speed (mph)', fontsize = 16)
plt.xlim (-1, 31)

# Add light grid with axis line for equator and notations for North and South
plt.grid(color = 'silver', linestyle = '-.', linewidth = 0.5)
plt.axhline(linewidth = 2, color = 'coral')
plt.text (31.5, -1.25, 'Equator', fontsize = 13, color = 'coral', style = 'italic')
plt.text (31.5, 80, 'North', fontsize = 13, color = 'navy')
plt.text (31.5, -83, 'South', fontsize = 13, color = 'navy')

# Show colorbar
plt.colorbar(orientation = 'horizontal')
plt.savefig((f'Wind Speed by Latitude {called_on}.png'))

plt.show()

In [ ]:
## can I figure out how to adjust temp range to just 125 rather than the 150 below? can I manipulate enough to use? 

fig = plt.figure(figsize = (15, 15))
ax = fig.add_subplot(111, projection = 'polar')
ax.set_ybound(0,120)
c = ax.scatter(city_weather_df['Latitude'], 
               city_weather_df['Temperature (F)'], 
               c=city_weather_df['Temperature (F)'], 
               s = 300, 
               edgecolor = 'teal', 
               alpha = 0.75)


plt.show()